# Gold Meta-Model Training - Attempt 2

**Architecture**: XGBoost Regression (reg:squarederror)

**Key Improvements from Attempt 1**:
- Aggressive regularization (max_depth 2-4, high L1/L2, low subsample)
- Direct overfitting penalty in Optuna objective (penalize train-val DA gap > 10pp)
- Position-change cost only (not daily cost)
- NO directional-weighted MAE
- NO price-level or CNY features
- 80 Optuna trials (increased from 50)

**Self-contained**: Data load → Preprocessing → HPO → Training → Evaluation → Save

In [ ]:
# ============================================================
# 1. IMPORTS
# ============================================================
import pandas as pd
import numpy as np
import xgboost as xgb
import optuna
import json
import os
from datetime import datetime
from sklearn.metrics import mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

# Reproducibility
np.random.seed(42)

print(f"Training started: {datetime.now().isoformat()}")
print(f"XGBoost version: {xgb.__version__}")
print(f"Optuna version: {optuna.__version__}")

In [ ]:
# ============================================================# 2. DATA LOADING# ============================================================print("\n=== Loading Data ===")# Load the 3 pre-split CSVs (verified by datachecker)train_df = pd.read_csv('../input/gold-prediction-complete/meta_model_attempt_2_train.csv', index_col=0, parse_dates=True)val_df = pd.read_csv('../input/gold-prediction-complete/meta_model_attempt_2_val.csv', index_col=0, parse_dates=True)test_df = pd.read_csv('../input/gold-prediction-complete/meta_model_attempt_2_test.csv', index_col=0, parse_dates=True)print(f"Train: {train_df.shape}")print(f"Val:   {val_df.shape}")print(f"Test:  {test_df.shape}")print(f"\nTrain date range: {train_df.index[0]} to {train_df.index[-1]}")print(f"Val date range:   {val_df.index[0]} to {val_df.index[-1]}")print(f"Test date range:  {test_df.index[0]} to {test_df.index[-1]}")# Separate features and targetTARGET_COL = 'target_next_day_return'X_train = train_df.drop(columns=[TARGET_COL])y_train = train_df[TARGET_COL]X_val = val_df.drop(columns=[TARGET_COL])y_val = val_df[TARGET_COL]X_test = test_df.drop(columns=[TARGET_COL])y_test = test_df[TARGET_COL]print(f"\nFeature count: {X_train.shape[1]}")print(f"Features: {list(X_train.columns)}")# Verify feature countassert X_train.shape[1] == 22, f"Expected 22 features, got {X_train.shape[1]}"assert X_val.shape[1] == 22, f"Expected 22 features in val, got {X_val.shape[1]}"assert X_test.shape[1] == 22, f"Expected 22 features in test, got {X_test.shape[1]}"print("\n✓ Data loaded successfully")

In [ ]:
# ============================================================
# 3. EVALUATION METRICS
# ============================================================

def direction_accuracy(y_true, y_pred):
    """Direction accuracy (excludes zero predictions and actuals)"""
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    # Exclude zeros to avoid np.sign(0) = 0 problem
    mask = (y_true != 0) & (y_pred != 0)
    if mask.sum() == 0:
        return 0.0
    
    correct = (np.sign(y_true[mask]) == np.sign(y_pred[mask])).sum()
    return correct / mask.sum()

def high_confidence_direction_accuracy(y_true, y_pred, threshold_percentile=75):
    """Direction accuracy for high-confidence predictions (top 25%)"""
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    # Filter high-confidence predictions
    threshold = np.percentile(np.abs(y_pred), threshold_percentile)
    mask = (np.abs(y_pred) >= threshold) & (y_true != 0) & (y_pred != 0)
    
    if mask.sum() == 0:
        return 0.0
    
    correct = (np.sign(y_true[mask]) == np.sign(y_pred[mask])).sum()
    return correct / mask.sum()

def sharpe_ratio(y_true, y_pred):
    """
    Sharpe ratio with position-change cost (5bps per trade).
    Matches CLAUDE.md specification.
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    # Trading positions based on prediction sign
    positions = np.sign(y_pred)
    
    # Count position changes (prepend 0 for first position)
    trades = np.abs(np.diff(positions, prepend=0))
    
    # Strategy returns = position * actual return - transaction cost
    strategy_returns = positions * y_true - trades * 0.0005  # 5bps per trade
    
    if len(strategy_returns) == 0 or np.std(strategy_returns) == 0:
        return 0.0
    
    # Annualized Sharpe (252 trading days)
    sharpe = np.mean(strategy_returns) / np.std(strategy_returns) * np.sqrt(252)
    return sharpe

def compute_all_metrics(y_true, y_pred, split_name=''):
    """Compute all evaluation metrics"""
    mae = mean_absolute_error(y_true, y_pred)
    da = direction_accuracy(y_true, y_pred)
    hcda = high_confidence_direction_accuracy(y_true, y_pred)
    sharpe = sharpe_ratio(y_true, y_pred)
    
    metrics = {
        'mae': mae,
        'direction_accuracy': da,
        'high_confidence_da': hcda,
        'sharpe_ratio': sharpe
    }
    
    if split_name:
        print(f"\n{split_name} Metrics:")
        print(f"  MAE: {mae:.4f}%")
        print(f"  Direction Accuracy: {da*100:.2f}%")
        print(f"  High-Confidence DA: {hcda*100:.2f}%")
        print(f"  Sharpe Ratio: {sharpe:.3f}")
    
    return metrics

print("✓ Evaluation functions defined")

In [ ]:
# ============================================================
# 4. OPTUNA HYPERPARAMETER OPTIMIZATION
# ============================================================

def optuna_objective(trial):
    """
    Optuna objective with weighted composite metric and overfitting penalty.
    
    Composite = 0.50*Sharpe + 0.30*DA + 0.10*(1-MAE) + 0.10*HCDA - overfitting_penalty
    
    Overfitting penalty: penalize train-val DA gap > 10pp
    """
    # Aggressive regularization ranges
    params = {
        'objective': 'reg:squarederror',
        'eval_metric': 'mae',
        'max_depth': trial.suggest_int('max_depth', 2, 4),
        'min_child_weight': trial.suggest_int('min_child_weight', 10, 30),
        'reg_lambda': trial.suggest_float('reg_lambda', 3.0, 20.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1.0, 10.0),
        'subsample': trial.suggest_float('subsample', 0.4, 0.7),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.3, 0.6),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.05, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'random_state': 42,
        'n_jobs': -1,
        'verbosity': 0
    }
    
    # Train model
    model = xgb.XGBRegressor(**params)
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        verbose=False
    )
    
    # Predictions
    train_pred = model.predict(X_train)
    val_pred = model.predict(X_val)
    
    # Validation metrics
    val_mae = mean_absolute_error(y_val, val_pred)
    val_da = direction_accuracy(y_val, val_pred)
    val_hcda = high_confidence_direction_accuracy(y_val, val_pred)
    val_sharpe = sharpe_ratio(y_val, val_pred)
    
    # Train metrics (for overfitting detection)
    train_da = direction_accuracy(y_train, train_pred)
    
    # Normalize metrics to [0, 1] range for composite
    sharpe_normalized = np.clip((val_sharpe + 1.0) / 3.0, 0, 1)  # assume Sharpe in [-1, 2]
    da_normalized = val_da  # already in [0, 1]
    mae_normalized = np.clip(val_mae / 2.0, 0, 1)  # assume MAE in [0, 2]
    hcda_normalized = val_hcda  # already in [0, 1]
    
    # Overfitting penalty: penalize train-val DA gap > 10pp
    da_gap_pp = (train_da - val_da) * 100
    overfitting_penalty = max(0, (da_gap_pp - 10) * 0.05)
    
    # Weighted composite (weights from design doc)
    composite = (
        0.50 * sharpe_normalized +
        0.30 * da_normalized +
        0.10 * (1 - mae_normalized) +
        0.10 * hcda_normalized -
        overfitting_penalty
    )
    
    # Log metrics for monitoring
    trial.set_user_attr('val_mae', val_mae)
    trial.set_user_attr('val_da', val_da)
    trial.set_user_attr('val_hcda', val_hcda)
    trial.set_user_attr('val_sharpe', val_sharpe)
    trial.set_user_attr('train_da', train_da)
    trial.set_user_attr('da_gap_pp', da_gap_pp)
    trial.set_user_attr('overfitting_penalty', overfitting_penalty)
    trial.set_user_attr('composite', composite)
    
    return composite

print("✓ Optuna objective function defined")

In [ ]:
# ============================================================
# 5. RUN HYPERPARAMETER SEARCH
# ============================================================
print("\n=== Starting Hyperparameter Optimization ===")
print("Configuration:")
print("  Trials: 80")
print("  Objective: Weighted composite (50% Sharpe, 30% DA, 10% MAE, 10% HCDA) - overfitting penalty")
print("  Pruner: MedianPruner")
print("  Regularization: AGGRESSIVE (max_depth 2-4, high L1/L2, low subsample)")

# Create Optuna study
study = optuna.create_study(
    direction='maximize',
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=5),
    sampler=optuna.samplers.TPESampler(seed=42)
)

# Run optimization
study.optimize(optuna_objective, n_trials=80, show_progress_bar=True)

print("\n=== Optimization Complete ===")
print(f"Best composite score: {study.best_value:.4f}")
print(f"\nBest hyperparameters:")
for key, value in study.best_params.items():
    print(f"  {key}: {value}")

print(f"\nBest trial metrics:")
best_trial = study.best_trial
print(f"  Val MAE: {best_trial.user_attrs['val_mae']:.4f}%")
print(f"  Val DA: {best_trial.user_attrs['val_da']*100:.2f}%")
print(f"  Val HCDA: {best_trial.user_attrs['val_hcda']*100:.2f}%")
print(f"  Val Sharpe: {best_trial.user_attrs['val_sharpe']:.3f}")
print(f"  Train DA: {best_trial.user_attrs['train_da']*100:.2f}%")
print(f"  DA Gap: {best_trial.user_attrs['da_gap_pp']:.2f}pp")
print(f"  Overfitting Penalty: {best_trial.user_attrs['overfitting_penalty']:.4f}")

In [ ]:
# ============================================================
# 6. TRAIN FINAL MODEL WITH BEST HYPERPARAMETERS
# ============================================================
print("\n=== Training Final Model ===")

best_params = study.best_params.copy()
best_params.update({
    'objective': 'reg:squarederror',
    'eval_metric': 'mae',
    'random_state': 42,
    'n_jobs': -1,
    'verbosity': 0
})

final_model = xgb.XGBRegressor(**best_params)
final_model.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_val, y_val)],
    verbose=True
)

print("\n✓ Final model trained")

In [ ]:
# ============================================================
# 7. EVALUATE ON ALL SPLITS
# ============================================================
print("\n=== Final Evaluation ===")

# Generate predictions
train_pred = final_model.predict(X_train)
val_pred = final_model.predict(X_val)
test_pred = final_model.predict(X_test)

# Compute metrics for all splits
train_metrics = compute_all_metrics(y_train, train_pred, 'TRAIN')
val_metrics = compute_all_metrics(y_val, val_pred, 'VAL')
test_metrics = compute_all_metrics(y_test, test_pred, 'TEST')

# Overfitting analysis
print("\n=== Overfitting Analysis ===")
print(f"Train-Val DA Gap: {(train_metrics['direction_accuracy'] - val_metrics['direction_accuracy'])*100:.2f}pp")
print(f"Train-Val Sharpe Gap: {train_metrics['sharpe_ratio'] - val_metrics['sharpe_ratio']:.3f}")
print(f"Train-Val MAE Gap: {train_metrics['mae'] - val_metrics['mae']:.4f}%")

In [ ]:
# ============================================================
# 8. FEATURE IMPORTANCE
# ============================================================
print("\n=== Feature Importance (Top 15) ===")

feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': final_model.feature_importances_
}).sort_values('importance', ascending=False)

print(feature_importance.head(15).to_string(index=False))

In [ ]:
# ============================================================
# 9. SAVE RESULTS
# ============================================================
print("\n=== Saving Results ===")

# Save model
final_model.save_model('model.json')
print("✓ Model saved to model.json")

# Save predictions
predictions_df = pd.DataFrame({
    'date': test_df.index,
    'actual': y_test.values,
    'predicted': test_pred
})
predictions_df.to_csv('predictions.csv', index=False)
print("✓ Predictions saved to predictions.csv")

# Save feature importance
feature_importance.to_csv('feature_importance.csv', index=False)
print("✓ Feature importance saved to feature_importance.csv")

# Save training results
training_result = {
    'feature': 'meta_model',
    'attempt': 2,
    'timestamp': datetime.now().isoformat(),
    'architecture': 'XGBoost Regression (reg:squarederror)',
    'best_params': best_params,
    'optuna_config': {
        'n_trials': 80,
        'best_composite_score': study.best_value,
        'direction': 'maximize',
        'objective': 'Weighted composite (50% Sharpe, 30% DA, 10% MAE, 10% HCDA) - overfitting penalty'
    },
    'metrics': {
        'train': train_metrics,
        'val': val_metrics,
        'test': test_metrics
    },
    'overfitting_analysis': {
        'train_val_da_gap_pp': (train_metrics['direction_accuracy'] - val_metrics['direction_accuracy']) * 100,
        'train_val_sharpe_gap': train_metrics['sharpe_ratio'] - val_metrics['sharpe_ratio'],
        'train_val_mae_gap': train_metrics['mae'] - val_metrics['mae']
    },
    'data_info': {
        'train_samples': len(X_train),
        'val_samples': len(X_val),
        'test_samples': len(X_test),
        'n_features': X_train.shape[1],
        'feature_names': list(X_train.columns)
    }
}

with open('training_result.json', 'w') as f:
    json.dump(training_result, f, indent=2, default=str)
print("✓ Training results saved to training_result.json")

print("\n=== Training Complete ===")
print(f"Finished: {datetime.now().isoformat()}")
print(f"\nFinal Test Metrics:")
print(f"  Direction Accuracy: {test_metrics['direction_accuracy']*100:.2f}%")
print(f"  High-Confidence DA: {test_metrics['high_confidence_da']*100:.2f}%")
print(f"  MAE: {test_metrics['mae']:.4f}%")
print(f"  Sharpe Ratio: {test_metrics['sharpe_ratio']:.3f}")